In [1]:
import numpy as np
import darkopt as dark # can import either entire package
from darkopt import materials, optimize # or import submodules seperately 
import darkopt.plotting as plot

### Optimize 1% coverage device

In [30]:
desired_coverage = 0.01

l_tes0 = 100e-6
l_tes_bnds = [5e-6, 300e-6]
l_overlap0 = 10e-6
l_overlap_bnds = [5e-06, 40e-6]
l_fin0 = 100e-6
l_fin_bnds = [25e-6, 500e-6]
n_fin0 = 4
n_fin_bnds = [2,6]

res_n = 320e-3
rp = 2e-3
ahole = 4.9e-11

bounds=[l_tes_bnds,l_overlap_bnds ,l_fin_bnds , n_fin_bnds]
fixrn=True

det_opt_1, eres, params = optimize.optimize_detector( tes_length0=l_tes0,
                                                    tes_l_overlap0=l_overlap0,
                                                    l_fin0=l_fin0,
                                                    n_fin0=n_fin0,
                                                    per_Al = desired_coverage,   
                                                    fixrn=fixrn,
                                                    #tes_width=w,
                                                    abs_type="Si", 
                                                    abs_shape="cube", 
                                                    abs_height=1e-2, 
                                                    abs_width=1e-2, 
                                                    w_safety=3e-3,
                                                    sigma=220000000.0,
                                                    rn=res_n,
                                                    rp=rp,
                                                    L_tot=5e-08,
                                                    ahole=ahole, 
                                                    con_type='ellipse',
                                                    veff_WAloverlap=.13,
                                                    bounds = bounds)

resolution: 224.0 [meV]
TES Length = 60.0 [μm]
Overlap Legth = 15.3 [μm]
Fin Length = 90.4 [μm]
Fin Height = 0.6 [μm]
N Fins = 4
Total Al surface coverage = 1.000 [%]
Absolute phonon collection energy efficiency = 25.64 [%]
Number of TESs = 181
Rn = 320.0 [mOhms]
Close Packed: False
---------------------------------
Phase margin = 492.9 [%] (phase_sep_legth - tes_legnth)/tes_legnth)


### Compare to Matt's optimum values

In [32]:
# Set some initial parameters
l = 50e-6 
w = 3.5e-06
l_overlap = 15e-6
n_fins = 4
sigma = 220000000.0
t0 = 40e-3 #41.5e-3
res_n = 320e-3
con_type= 'ellipse'
#con_type= 'modern'
l_f = 125e-6
h_fin = 600e-6
ahole = 4.9e-11

# create absorber 
absorber = dark.Absorber(name="Si", 
                         shape= "cube",
                         height= 1e-3, 
                         width=1e-2,  
                         w_safety=3e-3)
# create tes
tes = dark.TES(length=l,
               width=w, 
               l_overlap = l_overlap,
               n_fin=n_fins,
               sigma=sigma,
               rn=res_n,
               rsh = 5e-3,
               rp = rp,   
               L_tot= 5e-08, 
               material=materials.TESMaterial(Tc=t0),
               con_type=con_type, 
               operating_point=.35,
               veff_WAloverlap=.13)#0.45
      

# create qet
qet = dark.QET(l_fin=l_f, 
               h_fin=h_fin, 
               TES=tes, 
               ahole=ahole, 
               type_qp_eff=0, )

# create detector
det = dark.Detector(absorber, qet)
print(det.calc_res())
det.print(), det._close_packed

0.2152272164187309
---------------- DETECTOR PARAMETERS ----------------
nP =  1
SAactive =  5.874321123519332e-06
fSA_active =  0.009790535205865553
lcell =  0.00038490017945975053
SApassive =  4.2063619381653054e-07
fraction total Al cov =  0.010491595528893104
fSA_passive =  0.0007010603230275509
Alignment_area =  1.0385999999999999e-07
fSA_QPabsb =  0.010491595528893104
ePcollect =  0.933178864825957
tau_pabsb =  0.00013955445326783905
w_pabsb =  7165.661693939325
eE156 =  0.869
QP_eff =  0.45843847779328106
eEabsb =  0.19331656781965656
Kpb =  0.000155
nKpb =  4
NQET.TES =  108
total_L =  5e-08
------------------------------------------------



(None, False)

### TODO


* add in bonding pad area
* check how bias rail areas are calculated
